# Case Study: Improving marketing effectiveness

## Introduction

Most importantly, we do not expect you to answer everything perfectly. For example, it is not required to be an expert in user behavior/marketing analytics – the case is just an example to demonstrate your data skills. What we want to see is especially:

1- How easy is working with data sets for you? 
2- What methods do you know to prepare data and solve analytical issues? 
3- What do you focus on when analyzing data and what is of lesser priority to you?
4- How do you present in an easy-to-digest and actionable way? 

You have about 20 minutes to present the case study. After that, we will discuss the results and any questions from your side and our side. We are looking forward to hearing and seeing your findings!

> In order to generate more traffic and conversions, ImmoScout24 invests in paid advertising. The aim is to invest each € where it brings the most impact. Marketing managers would like to know whether they can improve budget allocation to marketing channels for one particular product. Moreover, they wonder if they could improve the way they currently assess the contribution of different channels.

-> How do the marketing channels currently perform? Has there been any change month over month? Estimate value of a conversion to be 100 €.
-> Can we improve the budget allocation based on last month’s performance? If yes, how should we shift the budgets? What would be the estimated effect?
-> Based on traffic data, is there another way we could consider the contribution of the different channels? If yes, show how this would be done and share your insights.


## Data Preparation

To address the marketing optimization questions, we can follow a structured approach involving data preparation, analysis, and presentation. 

#### Loading the data and understanding the columns

---
Costs dataset	

<img src="image-20231203-191906.png" width="100%" align="center" />

In [1]:
costs = _deepnote_execute_sql('SELECT *\nFROM \'costs_processed.csv\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
costs

,date,campaign_name,source_medium,impressions,clicks,spend
0,2020-04-24,de_br_reach_digitale_immobiliensuche,outbrain / display,0,0,0.00
1,2020-04-23,de_fi_generic_baufinanzierungsrechner_s_con_bu...,google / cpc,7486,1739,1941.71
2,2020-04-25,de_fi_generic_schufa_s_con_buyer_financecalc,google / cpc,361,51,174.87
3,2020-03-23,de_br_marke_+_stadt,bing / cpc,0,0,0.00
4,2020-04-05,de_br_marke,google / cpc,75928,44541,1468.00
...,...,...,...,...,...,...
2866,2020-03-27,de_br_marke_immobilien,bing / cpc,0,0,0.00
2867,2020-03-28,de_fi_kredit,bing / cpc,0,0,0.00
2868,2020-03-26,de_br_marke_bundesland,bing / cpc,0,0,0.00
2869,2020-03-21,de_fi_brand_baufinanzierung,google / cpc,0,0,0.00


-----
Traffic dataset	

<img src="image-20231203-191954.png" width="" align="" />

In [2]:
traffic = _deepnote_execute_sql('SELECT *\nFROM \'final_traffic.csv\'', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
traffic

,date,fullvisitorID,visitStartTime,channelGrouping,medium,source,isTrueDirect,hits,bounces,timeOnSite,conversion
0,2020-03-04,1.000339e+18,1583311404,Organic Search,organic,google,True,2,NaN,446.0,0
1,2020-03-05,1.000339e+18,1583399635,Organic Search,organic,google,True,4,NaN,489.0,0
2,2020-03-06,1.000339e+18,1583504222,Organic Search,organic,google,True,4,NaN,36.0,0
3,2020-03-06,1.000339e+18,1583520755,Organic Search,organic,google,True,1,1.0,NaN,0
4,2020-03-09,1.000339e+18,1583753297,Paid Search (Non-Brand),cpc,google,False,23,NaN,257.0,1
...,...,...,...,...,...,...,...,...,...,...,...
141419,2020-04-30,8.967218e+18,1588241835,Direct,(none),(direct),False,55,NaN,2194.0,0
141420,2020-04-30,9.063905e+18,1588268232,Paid Search (Non-Brand),cpc,google,False,20,NaN,229.0,0
141421,2020-04-30,9.125349e+18,1588235466,Referral,referral,deref-gmx.net,False,42,NaN,705.0,0
141422,2020-04-30,9.203921e+17,1588223800,Paid Search (Non-Brand),cpc,google,False,35,NaN,192.0,0


### Merging datasets and calculating metrics
Define the granularity, primary key, and columns that can be dropped without any loss.

-> The questions refer to performance of marketing channels by date, so the data should be aggregated by date and source.
-> The primary key (unique identifier for each row) is the date + marketing channel fields. 
-> The columns that are on the user or session level can be dropped. The analysis only goes until the channel level, so information regarding campaing can also be removed. 

In [55]:
funnel = _deepnote_execute_sql('WITH cs AS (\n    SELECT\n        date \n        , SPLIT_PART(source_medium, \' / \', 1) as source\n        , SPLIT_PART(source_medium, \' / \', 2) as medium\n        , SUM(impressions) AS impressions\n        , SUM(clicks) AS clicks\n        , SUM(spend) AS spend\n    FROM costs\n    GROUP BY 1, 2,3\n)\n, tf AS (\n    SELECT\n        date\n        , source\n        , medium\n        , CASE WHEN medium = \'cpc\' THEN \'Paid Search\' \n               WHEN channelGrouping = \'(Other)\' THEN medium \n               ELSE channelGrouping END AS channel_grouping\n        , SUM(hits) AS hits\n        , SUM(timeOnSite) AS time_on_site\n        , SUM(bounces) AS bounces\n        , SUM(conversion) AS conversion\n        , COUNT(fullvisitorID) AS users\n        , COUNT(DISTINCT fullVisitorId) AS distinct_users\n        , COUNT(DISTINCT CASE WHEN conversion = 1 THEN fullvisitorID ELSE NULL END) AS distinct_conversions\n    FROM traffic\n    GROUP BY 1, 2,3,4\n    )   \n\nSELECT\n    date\n    , source\n    , channel_grouping\n    , medium\n    , users\n    , distinct_users\n    , distinct_conversions\n    , time_on_site\n    , COALESCE(impressions, 0) AS impressions\n    , COALESCE(clicks, 0) AS clicks\n    , COALESCE(conversion, 0) AS conversion\n\n    , COALESCE(hits, 0) AS hits\n    , COALESCE(bounces, 0) AS bounces\n    , COALESCE(spend, 0) AS spend\n    , conversion * 100 AS conversion_value\nFROM tf LEFT JOIN cs USING(date, source, medium)', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
funnel

,date,source,channel_grouping,medium,users,distinct_users,distinct_conversions,time_on_site,impressions,clicks,conversion,hits,bounces,spend,conversion_value
0,2020-03-09,google,Paid Search,cpc,338,245,88,199017.0,206281,119570,95,10821,13.0,7206.53,9500
1,2020-03-22,google,Paid Search,cpc,291,212,69,201250.0,148931,89510,69,10026,12.0,4376.55,6900
2,2020-04-24,google,Paid Search,cpc,206,160,75,181986.0,181233,107727,75,9083,6.0,4801.17,7500
3,2020-04-08,google,Paid Search,cpc,302,226,66,194972.0,165793,98372,70,9915,14.0,4489.34,7000
4,2020-03-31,google,Paid Search,cpc,281,208,57,172699.0,180753,107646,57,9209,9.0,4452.95,5700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,2020-04-30,immo-goettingen.com,Referral,referral,1,1,0,408.0,0,0,0,62,0.0,0.00,0
4196,2020-04-30,immosuchmaschine.de,Cooperation,cooperation,1,1,0,157.0,0,0,0,36,0.0,0.00,0
4197,2020-04-30,deref-gmx.net,Referral,referral,1,1,0,705.0,0,0,0,42,0.0,0.00,0
4198,2020-04-30,standalone_az,Email,email,8,8,0,3527.0,0,0,0,292,0.0,0.00,0


### Question 1
How do the marketing channels currently perform? Has there been any change month over month? 

In [4]:
revenue_by_channel = _deepnote_execute_sql('SELECT\n    source\n    , channel_grouping\n    , SUM(spend)::INT AS total_cost\n    , SUM(conversion) * 100 AS converted_revenue    \n    , SUM(impressions) AS total_impressions\n    , SUM(clicks) AS total_clicks\n    , SUM(clicks) / SUM(impressions) AS click_through_rate\n    , SUM(conversion) AS total_conversions\nFROM funnel\nGROUP BY 1,2\nHAVING total_conversions > 0\n', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
revenue_by_channel

,source,channel_grouping,total_cost,converted_revenue,total_impressions,total_clicks,click_through_rate,total_conversions
0,google,Paid Search,323478,486400,10875939,6458585,0.593842,4864
1,bing,Paid Search,36251,16900,826977,268208,0.324323,169
2,outbrain,Display,0,1700,0,0,NaN,17
3,facebook,Display,25207,72500,5463211,39937,0.007310,725
4,google,Organic Search,0,631500,0,0,NaN,6315
...,...,...,...,...,...,...,...,...
178,zasobygwp.pl,Referral,0,100,0,0,NaN,1
179,abg.de,Referral,0,100,0,0,NaN,1
180,mobile.ok.ru,Referral,0,100,0,0,NaN,1
181,portus-immobilien.de,Referral,0,100,0,0,NaN,1


----
Revenue source analysis

In [5]:
DeepnoteChart(revenue_by_channel, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"axis":{"grid":false,"title":"Value "},"sort":null,"type":"quantitative","field":"converted_revenue","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"y":{"axis":{"grid":false},"sort":null,"type":"nominal","field":"channel_grouping","scale":{"type":"linear"}},"color":{"datum":"Converted revenue"},"yOffset":{"datum":"Converted revenue"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"axis":{"grid":false},"sort":null,"type":"quantitative","field":"total_cost","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"aggregate":"sum","formatType":"numberFormatFromNumberType"},"y":{"axis":{"grid":false},"sort":null,"type":"nominal","field":"channel_grouping","scale":{"type":"linear"}},"color":{"datum":"Cost"},"yOffset":{"datum":"Cost"}}}]}],"title":"","config":{"legend":{"orient":"top"}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

<br>
<br>

> Upon analyzing the revenue distribution across different channels, a distinct pattern emerges – a **significant proportion of conversions originates from non-paid searches.** 
>
> This observation signifies a notable level of trust by the customer in Scout24. Importantly, the strategic advantage lies in the lack of direct advertising costs associated with these organic conversions. Individuals who arrive via non-paid searches typically have a specific purpose or intent. They actively seek information related to products, services, or specific topics, indicating an increased likelihood of conversion. 
>
>This user segment is often in the consideration or decision stage of the purchasing journey, rendering their engagement particularly valuable.

----

#### Paid search analysis

In [6]:
roas = _deepnote_execute_sql('SELECT \n    date::date AS date\n    , source\n    , SUM(conversion_value) AS conversion_value\n    , SUM(spend) AS spend\n    , (SUM(conversion_value)/SUM(spend))::DECIMAL(10,2) AS return_on_ad_spend\n    , SUM(impressions) AS impressions\n    , SUM(clicks) AS clicks\n    , SUM(conversion) AS conversions\nFROM funnel\nWHERE channel_grouping = (\'Paid Search\') OR (source = (\'facebook\') AND channel_grouping = (\'Display\'))\nGROUP BY 1,2', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
roas

,date,source,conversion_value,spend,return_on_ad_spend,impressions,clicks,conversions
0,2020-03-09,google,9500,7206.53,1.32,206281,119570,95
1,2020-03-22,google,6900,4376.55,1.58,148931,89510,69
2,2020-04-24,google,7500,4801.17,1.56,181233,107727,75
3,2020-04-08,google,7000,4489.34,1.56,165793,98372,70
4,2020-03-31,google,5700,4452.95,1.28,180753,107646,57
...,...,...,...,...,...,...,...,...
175,2020-04-26,bing,0,0.00,NaN,0,0,0
176,2020-03-12,facebook,1000,649.88,1.54,134393,851,10
177,2020-04-23,facebook,200,227.69,0.88,22666,293,2
178,2020-04-22,bing,100,0.00,NaN,0,0,1


In [7]:
period_roas = _deepnote_execute_sql('SELECT \n    source\n    , SUM(spend) AS spend\n    , SUM(conversion_value) AS converted\n    , (SUM(conversion_value)/SUM(spend))::DECIMAL(10,2) AS roas\nFROM roas\nGROUP BY 1', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
period_roas

,source,spend,converted,roas
0,google,323478.170000,486400,1.50
1,bing,36250.900000,16900,0.47
2,facebook,25206.720002,72500,2.88


> The Return on Advertising Spend (ROAS) for each of the paid source media for the giving period is as follows:
1. Google / CPC: 1.50
2. Facebook/display: 2.88
3. Bing / CPC: 0.47

Looking at the general picture, Facebook's strategy provides the highest return on ad spend, closely followed by Google's strategy. Bing's strategy returns less than half of the investment, indicating a possible need for strategy adjustment or reallocation of resources.

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=len(roas['source'].unique()), subplot_titles=roas['source'].unique())

for idx, source in enumerate(roas['source'].unique()):
    current_source_data = roas[roas['source'] == source][['conversion_value', 'spend', 'date']].sort_values('date')

    fig.add_trace(go.Bar(x=current_source_data['date'], y=current_source_data['conversion_value'], name='return', marker=dict(color='skyblue')), row=idx+1, col=1)
    fig.add_trace(go.Scatter(x=current_source_data['date'], y=current_source_data['spend'], name='invested', line=dict(color='darksalmon')), row=idx+1, col=1)

fig.update_layout(height=700, width=900, showlegend=False)
fig.show()

> The monthly analysis reveals a consistent positive return from Google and Facebook channels, confirming their effectiveness in generating revenue. However, as also identified with the ROAS data, Bing does not demonstrate a favorable return on investment. It appears that the underperformance of Bing may have been recognized, leading to an adjustment in the budget. 
Notably, there is a significant reduction in investments in Bing starting from April onward. This strategic shift suggests a proactive response to the observed lower returns, aligning the budget with more effective channels and optimizing resource allocation for improved overall performance.

### Question 2
Can we improve the budget allocation based on last month’ performance? If yes, how should we shift the budgets? What would be the estimated effect?

In [46]:
df_roas = _deepnote_execute_sql('WITH base AS (\n    SELECT DISTINCT\n        date_trunc(\'month\', date) AS month\n        , source\n        , SUM(spend)::INT AS source_budget\n        , SUM(conversion_value)::INT AS source_revenue\n        , (SUM(conversion_value)/SUM(spend))::DECIMAL(10,2) AS roas\n    FROM roas \n    GROUP BY 1,2\n    ORDER BY source, month\n)\n\n, totals AS (\n    SELECT \n        month\n        , SUM(source_budget) AS total_budget\n        , SUM(source_revenue)  AS total_revenue\n        , SUM(roas)::DECIMAL(10,2) AS total_roas\n    FROM base\n    GROUP BY 1\n)\n\nSELECT \n    source\n    , roas\n    , source_budget\n    , (total_budget * (roas / total_roas))::int AS proposed_budget\n    , source_revenue\n    , (proposed_budget * roas)::INT AS new_revenue\n    , (new_revenue - source_revenue)::INT AS estimated_effect\n\nfrom base JOIN totals USING(month)\nWHERE month = (SELECT MAX(month) FROM totals)', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_roas

,source,roas,source_budget,proposed_budget,source_revenue,new_revenue,estimated_effect
0,bing,0.73,2337,21564,1700,15742,14042
1,facebook,3.41,7819,100729,26700,343486,316786
2,google,1.65,160877,48740,266100,80421,-185679


> The proposed budget allocation based on the last month's ROAS performance suggests the following changes:

1. For Bing, the proposed budget is 21,563, an increase from 2,337. This budget shift will result in an estimated effect (increase in conversion value) of approximately 14,035.

2. For Facebook, the proposed budget is 100,729, an increase from 7,819. This budget shift will result in an estimated effect (increase in conversion value) of approximately 316,824.

3. For Google, the proposed budget is 48,739, a decrease from 160,877. This budget shift will result in an estimated effect (decrease in conversion value) of approximately -185,026.

Note that this assumes that the increase or decrease in spending will result in proportional changes in conversion value. However, in reality, factors such as market saturation, diminishing returns, and changes in consumer behavior may affect the actual results. 

The proposed shifts in budget allocation aim to optimize the return on ad spend by investing more in channels with high ROAS (Facebook and Bing in this case) and less in channels with lower ROAS (Google in this case). It is important to continuously monitor the performance of these channels to ensure the effectiveness of these changes. The market and consumer behavior are dynamic and as such, the optimal budget allocation may change over time.

In [47]:
df_8 = _deepnote_execute_sql('SELECT\n    SUM(source_revenue) AS original_revenue\n    , SUM(new_revenue) AS simulated_revenue\n    , SUM(source_budget) AS original_budget\n    , SUM(proposed_budget) AS proposed_budget\nFROM df_roas', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_8

,original_revenue,simulated_revenue,original_budget,proposed_budget
0,294500,439649,171033,171033


### Question 3
Based on traffic data, is there another way how we could consider the contribution of the different channels? If yes, show how this would be done and share your insights.

To further evaluate the contribution of different channels, we could examine additional metrics apart from the simple conversion value or count. 

- Conversion rate: Number of conversions divided by the total number of visitors (conversions/visitors). It gives an idea of how successful the channel is in driving conversions relative to the number of users it attracts.
- Bounce rate: Number of visitors who leave the site after viewing just one page (bounces/visits). A high bounce rate might indicate that users brought in by the channel are not finding what they expect or need on the site.
 - Time on site: Estimate of user engagement. If users from a certain channel spend more time on the site, they could be more interested in the content or offerings, and possibly more likely to convert.
- Impression per user / Clicks per user:  These metrics can tell us, on average, how many ads a user from each source saw (impressions per user) and how many of them they clicked on (clicks per user). This can help us evaluate the user engagement for each,

In [77]:
additional_metrics = _deepnote_execute_sql('SELECT\n    source\n    , sum(conversion)/sum(users) AS conversion_rate\n    , sum(bounces)/sum(users) AS bounce_rate\n    , sum(time_on_site)/sum(users) AS time_on_site\n    , avg(impressions)/sum(users) impressions_per_user\n    , avg(clicks)/sum(users) AS clicks_per_user\n    , sum(users) AS total_users\n    , sum(distinct_users) AS distinct_users\n    , sum(conversion) AS conversions\n    , sum(bounces) AS bounces\nFROM funnel\nGROUP BY 1\nHAVING sum(conversion) > 0\n', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
additional_metrics

,source,conversion_rate,bounce_rate,time_on_site,impressions_per_user,clicks_per_user,total_users,distinct_users,conversions,bounces
0,google,0.176064,0.058100,740.664094,1.404023,0.833767,63494,42203,11179,3689.0
1,bing,0.193630,0.043069,959.899023,2.515160,0.815725,2763,2013,535,119.0
2,outbrain,0.037363,0.182418,364.824176,0.000000,0.000000,455,419,17,83.0
3,facebook,0.728614,0.057030,413.582104,51.652778,0.377591,1017,921,741,58.0
4,android,0.088032,0.236400,254.655786,0.000000,0.000000,1011,741,89,239.0
...,...,...,...,...,...,...,...,...,...,...
168,euroweb.net,1.000000,0.000000,3231.000000,0.000000,0.000000,1,1,1,0.0
169,ruhrgrund-immobilien.de,0.166667,0.000000,581.666667,0.000000,0.000000,12,8,2,0.0
170,zasobygwp.pl,0.142857,0.000000,116.571429,0.000000,0.000000,7,5,1,0.0
171,abg.de,1.000000,0.000000,606.000000,0.000000,0.000000,1,1,1,0.0


The calculated data shows the average number of impressions and clicks per user for different sources. A higher 'impressions_per_user' number means that a user from that source tends to see more ads, while a higher 'clicks_per_user' number means a user from that source tends to click on more ads. However, it appears that for many sources the average impressions and clicks per user are quite low, possibly due to a high number of users but low ad exposure or engagement.
While we have focused on traffic data for this analysis and in answering the previous questions, we could extend this technique to different aspects of the business such as customer segmentation, product performance, and regional performance, among others. This will provide a more comprehensive picture of the business's performance and allow for more effective decision-making.

> build a correlation matrix from additional_metrics 

In [82]:
corr

,conversion_rate,bounce_rate,time_on_site,impressions_per_user,clicks_per_user,total_users,distinct_users,conversions,bounces
conversion_rate,1.000000,-0.138168,0.301210,0.113179,0.047601,0.009127,0.009926,0.018171,0.004262
bounce_rate,-0.138168,1.000000,-0.221338,-0.010859,-0.018364,-0.004622,-0.005088,-0.007554,0.009990
time_on_site,0.301210,-0.221338,1.000000,-0.014131,0.010095,-0.002580,-0.002037,-0.000517,-0.012766
impressions_per_user,0.113179,-0.010859,-0.014131,1.000000,0.355653,0.035446,0.041663,0.082463,0.023331
clicks_per_user,0.047601,-0.018364,0.010095,0.355653,1.000000,0.635453,0.650815,0.662040,0.461485
total_users,0.009127,-0.004622,-0.002580,0.035446,0.635453,1.000000,0.998914,0.994238,0.917968
distinct_users,0.009926,-0.005088,-0.002037,0.041663,0.650815,0.998914,1.000000,0.994305,0.901849
conversions,0.018171,-0.007554,-0.000517,0.082463,0.662040,0.994238,0.994305,1.000000,0.887214
bounces,0.004262,0.009990,-0.012766,0.023331,0.461485,0.917968,0.901849,0.887214,1.000000


In [94]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate correlations
corr = additional_metrics.corr()
corr.conversion_rate.sort_values(ascending=False)

conversion_rate         1.000000
time_on_site            0.301210
impressions_per_user    0.113179
clicks_per_user         0.047601
conversions             0.018171
distinct_users          0.009926
total_users             0.009127
bounces                 0.004262
bounce_rate            -0.138168
Name: conversion_rate, dtype: float64

From the Correlation Matrix, we can derive the following insights:

1. 'Conversion Rate' is strongly positively correlated with 'Time on Site'. This implies that users spending more time on the site are more likely to convert. This makes intuitive sense as users who spend more time on the site are likely more engaged and thus more likely to make a purchase or complete a desired action.

2. 'Impressions Per User' has a moderate positive correlation with 'Conversion Rate'. This implies that as the user views more ads, they tend to click on ads more often.

2. 'Bounce Rate' has a strong negative correlation with 'Time on Site'. This means that the longer users stay on the site, the less likely they are to bounce.


This correlation matrix provides insights into the relationships between different metrics, which can help in understanding user behavior and creating effective marketing strategies. For example, strategies can be made to increase user engagement and time spent on the site to increase conversions, or efforts can be made to reduce bounce rates, which can also potentially lead to higher conversions.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e6cabd9c-7692-40d3-ab2f-2445a60671be' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>